## Imports and Setup

In [1]:
import os
import sys
import joblib
import subprocess

import pandas as pd
import numpy as np
import geopandas as gpd
from tqdm.notebook import tqdm

sys.path.insert(0, "../utils/")
import pred_utils
import config
import geoutils
import eval_utils
import fusion_utils

import logging
logging.basicConfig(level=logging.INFO)
pd.set_option('mode.chained_assignment', None)

%load_ext autoreload
%autoreload 2

C:\Users\Issa\miniconda3\envs\GFDRR\lib\site-packages\groundingdino\models\GroundingDINO\ms_deform_attn.py:31: UserWarning: Failed to load custom C++ ops. Running on CPU mode Only!
  warnings.warn("Failed to load custom C++ ops. Running on CPU mode Only!")


## Generate Post-disaster Predictions

In [2]:
filename = '../data/vectors/bldgs_ortho_LCA.gpkg'
bldgs = gpd.read_file(filename).reset_index(drop=True)[['UID', 'geometry']].dropna()
print(bldgs.shape)
bldgs.head(3)

ERROR:fiona._env:Pointer 'hGeom' is NULL in 'OGR_G_GetGeometryType'.



(69296, 2)


,UID,geometry
0,0.0,MULTIPOLYGON Z (((717760.366 1520511.462 0.000...
1,1.0,MULTIPOLYGON Z (((717797.038 1520510.844 0.000...
2,2.0,MULTIPOLYGON Z (((717926.809 1520525.188 0.000...


In [3]:
in_file = '../data/rasters/ortho/ortho_LCA.tif'
exp_config = '../configs/cnn/cnn-roof_material-inceptionv3-RGB_LCA.yaml'
c = config.load_config(exp_config, prefix='.')
bldgs = pred_utils.predict_image(bldgs, in_file, exp_config, prefix='.')
bldgs.head(3)

INFO:root:Config: {'attribute': 'roof_material', 'data': 'RGB_LCA', 'batch_size': 32, 'n_workers': 4, 'n_epochs': 60, 'model': 'inceptionv3', 'pretrained': True, 'resampler': 'RandomOverSampler', 'scheduler': 'ReduceLROnPlateau', 'optimizer': 'Adam', 'label_smoothing': 0.1, 'lr': 1e-05, 'img_size': 299, 'momentum': None, 'gamma': None, 'step_size': None, 'patience': 7, 'dropout': None, 'config_name': 'cnn-roof_material-inceptionv3-RGB_LCA', 'rasters_dir': '../data/rasters/', 'vectors_dir': '../data/vectors/', 'tile_dir': '../data/tiles/', 'csv_dir': '../data/csv/', 'out_dir': '../outputs/', 'exp_dir': '../exp/', 'log_dir': '../logs/'}
INFO:root:Model file ../exp/cnn-roof_material-inceptionv3-RGB_LCA\cnn-roof_material-inceptionv3-RGB_LCA.pth successfully loaded.
100%|███████████████| 69296/69296 [4:59:40<00:00,  3.85it/s]                                                           


,UID,geometry,ROOF_MATERIAL,ROOF_MATERIAL_PROB,INCOMPLETE_PROB,CONCRETE_CEMENT_PROB,HEALTHY_METAL_PROB,IRREGULAR_METAL_PROB
0,0.0,MULTIPOLYGON Z (((717760.366 1520511.462 0.000...,HEALTHY_METAL,0.628981,0.029425,0.046993,0.628981,0.294601
1,1.0,MULTIPOLYGON Z (((717797.038 1520510.844 0.000...,INCOMPLETE,0.904224,0.904224,0.027334,0.034043,0.034399
2,2.0,MULTIPOLYGON Z (((717926.809 1520525.188 0.000...,HEALTHY_METAL,0.916309,0.029696,0.031742,0.916309,0.022253


In [4]:
exp_config = '../configs/cnn/cnn-roof_type-efficientnetb0-RGB_LCA.yaml'
c = config.load_config(exp_config, prefix='.')
bldgs = pred_utils.predict_image(bldgs, in_file, exp_config, prefix='.')
bldgs.head(3)

INFO:root:Config: {'attribute': 'roof_type', 'data': 'RGB_LCA', 'batch_size': 32, 'n_workers': 4, 'n_epochs': 60, 'model': 'efficientnetb0', 'pretrained': True, 'resampler': 'RandomOverSampler', 'scheduler': 'ReduceLROnPlateau', 'optimizer': 'Adam', 'label_smoothing': 0.1, 'lr': 1e-05, 'img_size': 224, 'momentum': None, 'gamma': None, 'step_size': None, 'patience': 7, 'dropout': None, 'config_name': 'cnn-roof_type-efficientnetb0-RGB_LCA', 'rasters_dir': '../data/rasters/', 'vectors_dir': '../data/vectors/', 'tile_dir': '../data/tiles/', 'csv_dir': '../data/csv/', 'out_dir': '../outputs/', 'exp_dir': '../exp/', 'log_dir': '../logs/'}
INFO:root:Model file ../exp/cnn-roof_type-efficientnetb0-RGB_LCA\cnn-roof_type-efficientnetb0-RGB_LCA.pth successfully loaded.
100%|███████████████| 69296/69296 [2:42:01<00:00,  7.13it/s]                                                           


,UID,geometry,ROOF_MATERIAL,ROOF_MATERIAL_PROB,INCOMPLETE_PROB,CONCRETE_CEMENT_PROB,HEALTHY_METAL_PROB,IRREGULAR_METAL_PROB,ROOF_TYPE,ROOF_TYPE_PROB,NO_ROOF_PROB,FLAT_PROB,GABLE_PROB,HIP_PROB
0,0.0,MULTIPOLYGON Z (((717760.366 1520511.462 0.000...,HEALTHY_METAL,0.628981,0.029425,0.046993,0.628981,0.294601,HIP,0.898036,0.035153,0.026654,0.040157,0.898036
1,1.0,MULTIPOLYGON Z (((717797.038 1520510.844 0.000...,INCOMPLETE,0.904224,0.904224,0.027334,0.034043,0.034399,NO_ROOF,0.747590,0.747590,0.102935,0.028217,0.121258
2,2.0,MULTIPOLYGON Z (((717926.809 1520525.188 0.000...,HEALTHY_METAL,0.916309,0.029696,0.031742,0.916309,0.022253,GABLE,0.560529,0.023673,0.390195,0.560529,0.025602


In [5]:
name = 'ortho_LCA'
out_path = os.path.join(c['out_dir'])
if not os.path.isdir(out_path):
    os.makedirs(out_path)
out_file = os.path.join(out_path, f'{name}.gpkg')
bldgs.to_file(out_file, driver='GPKG')

In [6]:
bldgs.ROOF_MATERIAL.value_counts()

ROOF_MATERIAL
HEALTHY_METAL      41169
IRREGULAR_METAL    19365
INCOMPLETE          5642
CONCRETE_CEMENT     3120
Name: count, dtype: int64

In [7]:
bldgs.ROOF_TYPE.value_counts()

ROOF_TYPE
GABLE      30253
FLAT       19287
HIP        15096
NO_ROOF     4660
Name: count, dtype: int64

## Generate Pre-disaster Predictions

In [ ]:
name = 'Colihat'
filename = '../data/vectors/bldgs_drone_colihaut_DOM.gpkg'
bldgs = gpd.read_file(filename).reset_index(drop=True)[['UID', 'geometry']]
bldgs.head(3)

In [ ]:
in_file = '../data/rasters/drone/drone_colihaut_DOM.tif'
exp_config = '../configs/cnn/cnn-roof_material-resnet50-RGB_DOM.yaml'
bldgs = pred_utils.predict_image(bldgs, in_file, exp_config, prefix='.')
bldgs.head(3)

In [6]:
out_path = os.path.join(c['out_dir'], c['config_name'])
if not os.path.isdir(out_path):
    os.makedirs(out_path)
out_file = os.path.join(out_path, f'{name}.gpkg')
bldgs.to_file(out_file, driver='GPKG')

## Model Evaluation

In [19]:
ytrue_file = '../data/vectors/building_footprints_annotated_COLIHAUT.gpkg'
ytrue = gpd.read_file(ytrue_file)[['UID', 'roof_type', 'roof_material', 'geometry']].dropna()
ytrue.roof_type = ytrue.roof_type.replace({'PYRAMID': 'HIP', 'HALF_HIP': 'HIP'})
print(f"Data dimensions: {ytrue.shape}")
print(ytrue.roof_type.value_counts())
print(ytrue.roof_material.value_counts())
ytrue.head(3)

Data dimensions: (373, 4)
roof_type
GABLE      247
FLAT        72
HIP         44
NO_ROOF     10
Name: count, dtype: int64
roof_material
HEALTHY_METAL      187
IRREGULAR_METAL    130
CONCRETE_CEMENT     46
INCOMPLETE          10
Name: count, dtype: int64


,UID,roof_type,roof_material,geometry
4,34876,NO_ROOF,INCOMPLETE,"POLYGON Z ((665060.888 1712479.780 35.786, 665..."
5,34887,GABLE,HEALTHY_METAL,"POLYGON Z ((665078.259 1712486.585 32.366, 665..."
6,34890,GABLE,IRREGULAR_METAL,"POLYGON Z ((665070.715 1712492.135 27.175, 665..."


In [18]:
ypred_file = f'../output/{name}-drone-20230616.gpkg'
ypred = gpd.read_file(ypred_file)[['UID', 'roof_type', 'roof_material', 'geometry']].dropna()
print(f"Data dimensions: {ypred.shape}")
print(ypred.roof_type.value_counts())
print(ypred.roof_material.value_counts())
ypred.head(3)

Data dimensions: (373, 4)
roof_type
GABLE      250
HIP         59
FLAT        54
NO_ROOF     10
Name: count, dtype: int64
roof_material
HEALTHY_METAL      204
IRREGULAR_METAL    113
CONCRETE_CEMENT     48
INCOMPLETE           7
BLUE_TARP            1
Name: count, dtype: int64


,UID,roof_type,roof_material,geometry
0,34876,NO_ROOF,CONCRETE_CEMENT,"POLYGON Z ((665060.888 1712479.780 35.786, 665..."
1,34887,GABLE,HEALTHY_METAL,"POLYGON Z ((665078.259 1712486.585 32.366, 665..."
2,34890,GABLE,HEALTHY_METAL,"POLYGON Z ((665070.715 1712492.135 27.175, 665..."


In [26]:
attribute = 'roof_type'
classes = geoutils.classes_dict[attribute]
cm = eval_utils.get_confusion_matrix(ytrue[attribute], ypred[attribute], classes)
eval_utils.evaluate(ytrue[attribute], ypred[attribute])

{'overall_accuracy': 89.27613941018767,
 'balanced_accuracy': 87.45253649858913,
 'f1_score_micro': 89.27613941018767,
 'f1_score': 86.36588137237665,
 'precision_score': 87.00772128060264,
 'recall_score': 87.45253649858913,
 'f1_per_class': array([80.95238095, 92.95774648, 81.55339806, 90.        ]),
 'precision_per_class': array([94.44444444, 92.4       , 71.18644068, 90.        ]),
 'recall_per_class': array([70.83333333, 93.52226721, 95.45454545, 90.        ])}

## Data Fusion

In [8]:
c = config.create_config("../configs/fusion_06.yaml")
c = {key: '.' + value if 'config' in key or 'dir' in key else value for key, value in c.items()}

c1 = config.create_config(c['config1'])
c1 = {key: '.' + value if 'dir' in key else value for key, value in c1.items()}
classes = geoutils.classes_dict[c1['attribute']]
exp_dir = os.path.join(c['exp_dir'], c1['exp_name'])
model1 = pred_utils.load_model(c1, exp_dir=exp_dir , n_classes=len(classes))
print(c1)

c2 = config.create_config(c['config2'])
exp_dir = os.path.join(c['exp_dir'], c2['exp_name'])
c2 = {key: '.' + value if 'dir' in key else value for key, value in c2.items()}
model2 = pred_utils.load_model(c2, exp_dir=exp_dir, n_classes=len(classes))
print(c2)

Model file ../exp/exp-09\best_model.pth successfully loaded.
{'data_dir': '../data/rasters/Tiles/ortho/', 'csv_dir': '../data/csv/', 'attribute': 'roof_material', 'exp_name': 'exp-09', 'mode': 'RGB', 'batch_size': 32, 'n_workers': 4, 'n_epochs': 30, 'model': 'efficientnetb0', 'pretrained': True, 'scheduler': 'ReduceLROnPlateau', 'optimizer': 'Adam', 'lr': 1e-05, 'img_size': 224, 'momentum': None, 'gamma': None, 'step_size': None, 'patience': 7, 'dropout': None}
Model file ../exp/exp-06\best_model.pth successfully loaded.
{'data_dir': '../data/rasters/Tiles/ndsm/', 'csv_dir': '../data/csv/', 'attribute': 'roof_material', 'exp_name': 'exp-06', 'mode': 'GRAYSCALE', 'batch_size': 32, 'n_workers': 4, 'n_epochs': 30, 'model': 'inceptionv3', 'pretrained': True, 'scheduler': 'ReduceLROnPlateau', 'optimizer': 'Adam', 'lr': 1e-05, 'img_size': 299, 'momentum': None, 'gamma': None, 'step_size': None, 'patience': 7, 'dropout': None}


In [101]:
source1 = '../data/rasters/Ortho/ortho_DOM.tif'
source2 = '../data/rasters/DSM/ndsm_DOM.tif'
data = fusion.predict(bldgs, c1, c2, model1, model2, source1=source1, source2=source2)

100%|███████████████| 522/522 [11:29<00:00,  1.32s/it]                                                                 


In [9]:
features = fusion.get_features(c, data)
model_file = os.path.join(c['exp_dir'], c['exp_name'], c['mode'], c['model'], 'best_model.pkl')
model = joblib.load(model_file)
preds = model.predict(data[features])

Pipeline(steps=[('scaler', MinMaxScaler()), ('selector', None),
                ('model',
                 RandomForestClassifier(bootstrap=False, criterion='entropy',
                                        max_depth=9, n_estimators=450,
                                        n_jobs=-1, random_state=42,
                                        verbose=1))])

In [104]:
bldgs['roof_material'] = preds
out_path = '../output/'
if not os.path.isdir(out_path):
    os.makedirs(out_path)
out_file = os.path.join(out_path, f'{name}-ortho-20230613.gpkg')
bldgs.to_file(out_file, driver='GPKG')